In [36]:
import sys
import numpy as np
import import_ipynb
from pathlib import Path
from scipy.sparse.linalg import spsolve

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps
    from pre_processing import create_domain, read_mesh
    from fem_processing import boundary_conditions, matrices_assembly
    from pos_processing import graph_results, fem_analysis
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Modules imports were successful!


# Project 2: Coaxial Cable with longitudinal symmetry and a quarter domain

Considere a seção reta transversal de um cabo coaxial singelo apresentado na Figura $(1)$.

<figure>
    <img src="pre_processing/pictures/coaxial_longitudinal.png" alt="Fig.1" style="width:20%;" />
    <figcaption>Figure 1: Geometry of Coaxial Problem.</figcaption>
</figure>

# 6-noded Quadratic Triangular Elements, $P_2$
## Pre-processor module
### Geometry and mesh Domain

In [37]:
FINITE_ELEMENT = ("Triangle", 2)

BOUNDARY = [{'tag': 101, 'type': 'Dirichlet',  'value': 0, 'name': 'outer_sheath'}]

MATERIAL = [{'tag': 201, 'name': 'core', 'relative_magnetic_permeability': 1},
            {'tag': 202, 'name': 'dielectric', 'relative_magnetic_permeability': 1},
            {'tag': 203, 'name': 'sheath', 'relative_magnetic_permeability': 1}]

# Criar geometria coaxial
create_domain.coaxial_quarter_longitudinal(FINITE_ELEMENT, BOUNDARY, MATERIAL, h=1E-3, view_mesh=False)

# Create mesh from file geometry
mesh_data = read_mesh.get_data(FINITE_ELEMENT, BOUNDARY, MATERIAL, model='coax_quarter', info_mode=False)

Malha salva em pre_processing/mesh/coax_quarter_domain_Triangle2.msh
Model coax_quarter_domain_Triangle2 (2D)
Info     : 19 geometric entities
Info     : 4 Physical Groups
Info     : 1675 nodes in total
Info     : 800 2-D elements in total


### Mesh Data Dictionaries

In [38]:
mesh_nodes = mesh_data['nodes_data']; mesh_nodes[:1]

[{'TagID': 1,
  'global_coord': (0.0, 0.0),
  'bc': {'type': 'Free', 'value': None}}]

In [39]:
conn_data = mesh_data['conn_data']; conn_data[:1]

[{'TagID': 126,
  'conn_list': [236, 287, 271, 316, 317, 318],
  'material': {'tag': 201,
   'name': 'core',
   'relative_magnetic_permeability': 1}}]

### ``Nodes`` and ``conn`` list

In [40]:
nodes = [node['global_coord'] for node in mesh_data['nodes_data']]
conn = [element['conn_list'] for element in mesh_data['conn_data']]

### Figure 1: Coaxial meshed domain 

In [41]:
graph_results.plot_mesh(mesh_data, FINITE_ELEMENT, model='coax_quarter', Numbering=False)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p21_coaxial_quarter_magnetostatics\pre_processing\pictures\coax_quarter_meshed_domain_Triangle2.svg


<figure>
    <img src="pre_processing/pictures/coax_quarter_meshed_domain_Triangle2.svg" alt="Fig.1" style="width:90%;" />
    <figcaption>Figure 1: Coaxial meshed domain.</figcaption>
</figure>

### Assembly Matrices
#### `assembly.global_nodes_coordinates()` 

In [42]:
ai, xi, yi = matrices_assembly.global_nodes_coordinates(0, mesh_data)
print(f"'a_1' Global coordinates: {ai}")
print(f"'x_1' x-coordinates: {xi}")
print(f"'y_1' y-coordinates: {yi}")

'a_1' Global coordinates: [(0.004569148134344793, 0.000909757283515022), (0.005532691729608888, 0.000864570036453122), (0.005117508794767472, 0.001888591451683152), (0.005050919931976841, 0.000887163659984072), (0.00532510026218818, 0.001376580744068137), (0.004843328464556133, 0.001399174367599087)]
'x_1' x-coordinates: [0.004569148134344793, 0.005532691729608888, 0.005117508794767472, 0.005050919931976841, 0.00532510026218818, 0.004843328464556133]
'y_1' y-coordinates: [0.000909757283515022, 0.000864570036453122, 0.001888591451683152, 0.000887163659984072, 0.001376580744068137, 0.001399174367599087]


#### `map_to_physical_coordinates()`

In [43]:
xi_master = (1, 0)
xg, yg = matrices_assembly.isomapping_to_global_coordinates(ai, xi_master, FINITE_ELEMENT)
print(f"'e_1' Master coordinates: {xi_master} --> Global coordinates: ({xg}, {yg})")

'e_1' Master coordinates: (1, 0) --> Global coordinates: (0.005532691729608888, 0.000864570036453122)


#### `apply_physics()`

In [44]:
mesh_data = ps.apply_physics(mesh_data, FINITE_ELEMENT)
mesh_data['conn_data'][:1]

[{'TagID': 126,
  'conn_list': [236, 287, 271, 316, 317, 318],
  'material': {'tag': 201,
   'name': 'core',
   'relative_magnetic_permeability': 1},
  'source': {'type': 'uniform_current_density_Jz', 'value': 3183.098861837907},
  'stiffness_a_value': 795774.7150262763}]

#### Material Proprieties, $k_a$

In [45]:
Ka = [cell['stiffness_a_value'] for cell in mesh_data['conn_data']]
Ma = [cell['mass_a_value'] for cell in mesh_data['conn_data'] if 'mass_a_value' in cell.keys()]
print("Stiffness Material values: ", Ka[:3])
print("Mass Material values: ", Ma[:3])

Stiffness Material values:  [795774.7150262763, 795774.7150262763, 795774.7150262763]
Mass Material values:  []


#### Source: Current density, $J_z$

In [46]:
Jz = [element['source']['value'] for element in mesh_data['conn_data']]
print("Source current density vector Jz: \n", Jz[:1])

Source current density vector Jz: 
 [3183.098861837907]


#### Jacobian Matrix Transform

In [47]:
Je = matrices_assembly.jacobian(e=0, mesh_data=mesh_data, element_type=FINITE_ELEMENT, xik=(0, 0)) 
Jdet, Jinv = np.linalg.det(Je), np.linalg.inv(Je)
print("Jacobian matrix for element e_0:\n", Je)
print("Determinant of the Jacobian matrix for element e_0:", Jdet)

Jacobian matrix for element e_0:
 [[ 9.63543595e-04 -4.51872471e-05]
 [ 5.48360660e-04  9.78834168e-04]]
Determinant of the Jacobian matrix for element e_0: 9.67928302205606e-07


#### Local Elements $e_1$

In [48]:
#Ae, fe, areae = matrices_assembly.linear_local_matrices(e=0, mesh_data=mesh_data)
Ae, fe, Me, area_e = matrices_assembly.higher_order_local_matrices(0, mesh_data, FINITE_ELEMENT)
print("Local stiffness matrix for element e_1:\n", Ae)
print("Local load vector for element e_1:\n", fe)
print("Area of element e_1: ", area_e)

Local stiffness matrix for element e_1:
 [[ 5.01916473e+05  1.06149131e+05  6.11563586e+04 -4.24596527e+05
  -3.29796204e-03 -2.44625434e+05]
 [ 1.06149131e+05  5.17462774e+05  6.63384606e+04 -4.24596526e+05
  -2.65353842e+05  2.69544082e-03]
 [ 6.11563586e+04  6.63384606e+04  3.82484456e+05  1.82320406e-03
  -2.65353842e+05 -2.44625435e+05]
 [-4.24596527e+05 -4.24596526e+05  1.82320406e-03  1.86915160e+06
  -4.89250864e+05 -5.30707690e+05]
 [-3.29796204e-03 -2.65353842e+05 -2.65353842e+05 -4.89250864e+05
   1.86915160e+06 -8.49193047e+05]
 [-2.44625434e+05  2.69544082e-03 -2.44625435e+05 -5.30707690e+05
  -8.49193047e+05  1.86915160e+06]]
Local load vector for element e_1:
 [[ 1.23388978e-12]
 [-2.09288085e-14]
 [-2.09288018e-14]
 [ 5.13501909e-04]
 [ 5.13501904e-04]
 [ 5.13501909e-04]]
Area of element e_1:  4.839641462631615e-07


#### Global “stiffness” matrix, $A_g$

In [49]:
Ag, fg, Mg = matrices_assembly.get_global_matrix(mesh_data, FINITE_ELEMENT)
print("Global matrix shape:", Ag.shape)

Global matrix shape: (1675, 1675)


### Imposition of Boundary Conditions

In [50]:
dirichlet_data = {node['TagID']: node['bc']['value'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet'}
Nu = len(mesh_data['nodes_data'])
Nd = len(dirichlet_data)
print(f"The entire domain has {Nu} nodes: {Nu - Nd} free nodes and {Nd} Dirichlet nodes.")

The entire domain has 1675 nodes: 1610 free nodes and 65 Dirichlet nodes.


### Mapping Global nodes to reduced system

This code creates a dictionary called ``global_to_reduced``, which maps the global indices of the mesh nodes to the reduced indices, i.e. the indices that correspond only to the nodes that are not in the _Dirichlet boundary conditions_.

In [51]:
global_to_reduced = {node: i for i, node in enumerate(
        idx_n for idx_n in range(1, len(nodes)+1) if idx_n not in dirichlet_data)}

## Processor Module
### Reduced Global matrix $A_{gr}$ with boundary conditions

In [52]:
Agr, fgr, Mgr = boundary_conditions.get_reduced_global_matrices(mesh_data, FINITE_ELEMENT)
print("Reduced global matrix shape:", Agr.shape)

Reduced global matrix shape: (1610, 1610)


### Global Potential Vector

In [53]:
ur = spsolve(Agr.tocsr(), fgr.toarray())
potential_u = boundary_conditions.get_global_potentials(mesh_data, ur)
print("Reduced solution vector ur shape: ", ur.shape)
print("Global potential vector V shape: ", potential_u.shape)

Reduced solution vector ur shape:  (1610,)
Global potential vector V shape:  (1675,)


### Solution at each element

In [54]:
u_list = []
for e in range(len(conn)):
    uh_values = [potential_u[id - 1] for id in conn[e]]
    u_list.append(uh_values)
print("The solution u for Node Element is: \n", np.array(u_list[:3]))

The solution u for Node Element is: 
 [[2.03229195e-07 1.93575810e-07 1.95178293e-07 1.98635117e-07
  1.94682301e-07 1.99518448e-07]
 [2.18187257e-07 2.22088271e-07 2.18313082e-07 2.20431544e-07
  2.20424467e-07 2.18570332e-07]
 [1.24932921e-07 1.46030644e-07 1.24932777e-07 1.35877450e-07
  1.35826846e-07 1.24934917e-07]]


## Post-Processor

In [55]:
graph_results.approximate_fem_solution(FINITE_ELEMENT, mesh_data, potential_u, model='coax_quarter')

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p21_coaxial_quarter_magnetostatics\pos_processing\pictures\coax_quarter_domain_solution_Triangle2.svg


### Figure 2: Scalar Field Distribution on a Triangular Mesh

<figure>
    <img src="pos_processing/pictures/coax_quarter_domain_solution_Triangle2.svg" alt="Fig.2" style="width:70%;" />
    <figcaption>Figure 2: Scalar Field Distribution on a Triangular Mesh.</figcaption>
</figure>

### Magnetic Energy Stored

In [56]:
analytical_solution = lambda x, y: ps.potential_and_gradient(x, y)
norm_l2, norm_energy, total_energy = fem_analysis.errors_and_energy(
            mesh_data, FINITE_ELEMENT, potential_u, analytical_solution)
print(f"Partial magnetic energy stored: {total_energy:.5} J.")
print(f"Total magnetic energy stored: {4*total_energy:.5} J.")

Partial magnetic energy stored: 2.1269e-08 J.
Total magnetic energy stored: 8.5076e-08 J.


### `magnetic_field(mesh_data, element_type, uh)`

In [57]:
B_field = fem_analysis.magnetic_field_density(mesh_data, FINITE_ELEMENT, potential_u)
graph_results.magnetic_field_distribution(mesh_data, B_field, FINITE_ELEMENT)
graph_results.radial_profile(mesh_data, B_field, FINITE_ELEMENT)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p21_coaxial_quarter_magnetostatics\pos_processing\pictures\b_field_Triangle2.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p21_coaxial_quarter_magnetostatics\pos_processing\pictures\radial_b_profile_Triangle2.svg


### Figure 3: Magnetic field density distribution

<figure>
    <img src="pos_processing/pictures/b_field_Triangle2.svg" alt="Fig.3" style="width:70%;" />
    <figcaption>Figure 3: Magnetic field density distribution.</figcaption>
</figure>

### Figure 4: Magnetic density field radial profile

<figure>
    <img src="pos_processing/pictures/radial_b_profile_Triangle2.svg" alt="Fig.4" style="width:70%;" />
    <figcaption>Figure 4: Magnetic field density radial profile.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.